Notes

This is going to be a logistic regression model

In [1]:
import pandas as pd
from util import get_data, set_cwd_to_script, dependent_variable, categorical_variables, continuous_variables
set_cwd_to_script()
pd.set_option('display.max_rows', 500)

Correlation matrix

Independent variables that correlate strongly with the dependent variable (action taken) should be included in the model.

Alot of the independent variables are correlated with each other. This is called multicolinarity and can interfere with the model results.
https://towardsdatascience.com/multi-collinearity-in-regression-fe7a2c1467ea



In [2]:
loan_data = get_data("state_IL_application.csv")
loan_data.isnull().sum()

activity_year                                    1
lei                                              1
derived_msa-md                                   1
state_code                                       1
county_code                                   2901
census_tract                                  3118
conforming_loan_limit                         3084
derived_loan_product_type                        1
derived_dwelling_category                        1
action_taken                                     1
purchaser_type                                   1
preapproval                                      1
loan_type                                        1
loan_purpose                                     1
lien_status                                      1
reverse_mortgage                                 1
open-end_line_of_credit                          1
business_or_commercial_purpose                   1
loan_amount                                      1
loan_to_value_ratio            

In [3]:
def process_categorical_variables(df, variable_list):
    # set types as category
    for variable in variable_list:
        df[variable] = df[variable].astype("category")
    return df

loan_data = process_categorical_variables(loan_data, categorical_variables)

KeyError: 'ageapplicant'

In [ ]:
loan_data.head(10)

In [ ]:
corr=loan_data.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr["absolute_correlation"] = corr["action_taken"].abs()
corr = corr.sort_values(by=["absolute_correlation"], ascending=False)
corr["absolute_correlation"]